In [1]:
import json
import os
import pandas as pd
import numpy as np
import requests as req
import random

## Reporte de ventas

In [16]:
# Crear un df con la columnas "fechas - pasado - final"
reporte = pd.DataFrame(columns=['fecha', 'Monto Pasado', 'Monto Actual'])
reporte.to_csv('reporte.csv', index=False)

## Configuración

In [23]:
# Datos 

dic_datos = {
    "fechas" : ["2024-10-09"],
    "montos_reporte": [23351],
    "monto_limite": 6000,
    "servidor" : "26.144.145.93",
    "productos" : ["034003", "042035"]
}

# Crear un directorio para guardar los datos

## API

In [20]:
api = f'http://{dic_datos["servidor"]}:8000/florcatorce/mantenimiento/'

## Obtener las ventas consumiento la API

In [24]:
i = 0
for fecha in dic_datos["fechas"]:
    ventas = req.get(f'{api}ventas', json={"fecha": fecha})
    # Crear un marco de trabajo
    df = pd.DataFrame(ventas.json())
    # Cambiar los tipos de datos
    df["efectivo"] = df["efectivo"].astype(float)
    df["tarjeta"] = df["tarjeta"].astype(float)
    df["total"] = df["total"].astype(float)
    # Obtener los folios para hacer mantenimiento
    df_folios = df[(df["efectivo"] >= 120) & (df["tarjeta"] == 0) & (df["facturado"] == False)]
    # Crear un array de folios
    folios_array = np.array(df_folios["folio"])
    random.shuffle(folios_array)
    total_folios = np.size(folios_array)
    # Crear un array de efectivo
    efectivo_array = np.array(df_folios["efectivo"])
    total_efectivo = np.sum(efectivo_array)
    # Abrir el archivo de reporte
    df_reporte = pd.read_csv('reporte.csv')
    # Empezar con el mantenimiento de ventas
    efectivo_folios = 0
    folios_afectados = []
    for folio in folios_array:
        producto_select = np.random.choice(dic_datos["productos"])
        try:
            update = req.patch(f'{api}{folio}', json={"producto": producto_select, "cantidad": 1})
            print(update.json())
            print('\n')
        except Exception as e:
            print(e)
            exit()
        efectivo_folio = df_folios[df_folios["folio"] == folio]["efectivo"].values[0]
        efectivo_folios += efectivo_folio
        
        if producto_select == "034003":
            efectivo_folios -= 80
        elif producto_select == "042035":
            efectivo_folios -= 40
        folios_afectados.append(folio)
        # Determinar si el proceso sigue o no
        flag = dic_datos["montos_reporte"][i] - efectivo_folios
        if flag > 6000 and flag < 7050:
            print(f'El mantenimiento acabo con {flag} en la fecha {fecha}')
            print(f'Folios afectados: {folios_afectados}')
            break
    i += 1

{'datos actualizados': {'venta': {'folio': 11830, 'fecha': '2024-10-09T21:54:34Z', 'salidarepartidor': None, 'cierre': '2024-10-09T21:57:34Z', 'mesa': 'P/LL', 'nopersonas': '1', 'cambio': '0.0000', 'descuento': '0.0000', 'facturado': False, 'cambiorepartidor': '0.0000', 'usuariodescuento': '', 'idtipodescuento': '0', 'propinapagada': False, 'propinafoliomovtocaja': '0', 'propinaincluida': '0.0000', 'propinamanual': False, 'totalarticulos': '1.00', 'subtotal': '80.0000', 'subtotalsinimpuestos': '80.0000', 'total': '80.0000', 'totalconpropina': '80.0000', 'totalimpuesto1': '80.0000', 'cargo': '0.0000', 'totalconcargo': '80.0000', 'totalconpropinacargo': '80.0000', 'descuentoimporte': '0.0000', 'efectivo': '80.0000', 'tarjeta': '0.0000', 'vales': '0.0000', 'otros': '0.0000', 'propina': '0.0000', 'propinatarjeta': '0.0000', 'totalsindescuento': '80.0000', 'totalalimentos': '0.0000', 'totalbebidas': '0.0000', 'totalotros': '80.0000', 'totaldescuentos': '0.0000', 'totaldescuentoalimentos': '

In [22]:
# Mostar cuantos folios se afectaron
afectado = np.size(folios_afectados)
print(afectado)

35
